In [1]:
#import necessary packages
import pandas as pd
from fuzzywuzzy import fuzz, process


C:\Users\giuli\AppData\Local\Programs\Python\Python310\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
#import data
#season 2019_20
PL2019_ratings = pd.read_csv('Data/2019-20/Players/PL2019_ratings.csv')
PL2019_fl = pd.read_csv('Data/2019-20/Players/FL_19.csv')
PL19_lineups = pd.read_csv('Data/2019-20/Players/PL2019_lineups.csv')
PL2019_fl.rename(columns={"round": "matchweek"}, inplace=True)
PL2019_fl['player'] = PL2019_fl['name']
#season 2020_21
PL2020_ratings = pd.read_csv('Data/2020-21/Players/PL2020_ratings.csv')
PL2020_fl = pd.read_csv('Data/2020-21/Players/FL_20.csv')
PL20_lineups = pd.read_csv('Data/2020-21/Players/PL2020_lineups.csv')
PL2020_fl.rename(columns={"round": "matchweek"}, inplace=True)
PL2020_fl['player'] = PL2020_fl['name']
#season 2020_21
PL2021_ratings = pd.read_csv('Data/2021-22/Players/PL2021_ratings.csv')
PL2021_fl = pd.read_csv('Data/2021-22/Players/FL_21.csv')
PL21_lineups = pd.read_csv('Data/2021-22/Players/PL2021_lineups.csv')
PL2021_fl.rename(columns={"round": "matchweek"}, inplace=True)
PL2021_fl['player'] = PL2020_fl['name']


In [52]:
PL2020_fl.replace(['Brighton', 'Leeds', 'Leicester', 'Man City', 'Man Utd', 'Newcastle', 'Sheffield Utd','Spurs','West Brom','West Ham', 'Wolves'], ['Brighton & Hove Albion', 'Leeds United', 'Leicester City','Manchester City', 'Manchester United', 'Newcastle United', 'Sheffield United','Tottenham Hotspur', 'West Bromwich Albion', 'West Ham United','Wolverhampton Wanderers'], inplace=True)
PL2020_ratings.loc[PL2020_ratings['position'] == 'GK', 'category'] = 'Goalkeeper'

In [53]:
PL2020_ratings.to_excel('ratings_check.xlsx')

In [38]:
players_rating = PL2020_ratings[['player', 'Team']]
players_rating = players_rating.drop_duplicates()
players_rating

,player,Team
0,Joe Bryan,Fulham
1,Tim Ream,Fulham
2,Michael Hector,Fulham
3,Denis Odoi,Fulham
4,Ainsley Maitland-Niles,Arsenal
...,...,...
10190,João Virgínia,Everton
10203,John Ruddy,Wolverhampton Wanderers
10338,Scott Carson,Manchester City
10345,Jack Butland,Crystal Palace


In [39]:
players_fl = PL2020_fl[['player', 'name', 'team']]
players_fl = players_fl.drop_duplicates()
players_fl

,player,name,team
0,Aaron Connolly,Aaron Connolly,Brighton & Hove Albion
1,Aaron Cresswell,Aaron Cresswell,West Ham United
2,Aaron Mooy,Aaron Mooy,Brighton & Hove Albion
3,Aaron Ramsdale,Aaron Ramsdale,Sheffield United
4,Abdoulaye Doucouré,Abdoulaye Doucouré,Everton
...,...,...,...
23955,William Fish,William Fish,Manchester United
24064,Sylvester Jasper,Sylvester Jasper,Fulham
24183,William Osula,William Osula,Sheffield United
24237,Tim Iroegbunam,Tim Iroegbunam,West Bromwich Albion


In [40]:
player_lineup = PL20_21_lineups[['player', 'team']]
player_lineup= PL20_21_lineups.drop_duplicates()
player_lineup['name_lineup'] = PL20_21_lineups['player']
player_lineup

,player,team,name_lineup
0,Marek Rodák,Fulham,Marek Rodák
1,Michael Hector,Fulham,Michael Hector
2,Denis Odoi,Fulham,Denis Odoi
3,Neeskens Kebano,Fulham,Neeskens Kebano
4,Tom Cairney,Fulham,Tom Cairney
...,...,...,...
14441,Hannibal Mejbri,Manchester United,Hannibal Mejbri
14442,William Thomas Fish,Manchester United,William Thomas Fish
14523,Tim Iroegbunam,West Bromwich Albion,Tim Iroegbunam
14541,Tyrese Francois,Fulham,Tyrese Francois


In [41]:
players_total = pd.merge(players_fl,player_lineup  , on='player', how='left')
#players_total['test'].isna().sum()
players_total = players_total.sort_values(by='name_lineup')

players_total

,player,name,team_x,team_y,name_lineup
0,Aaron Connolly,Aaron Connolly,Brighton & Hove Albion,Brighton & Hove Albion,Aaron Connolly
1,Aaron Cresswell,Aaron Cresswell,West Ham United,West Ham United,Aaron Cresswell
3,Aaron Ramsdale,Aaron Ramsdale,Sheffield United,Sheffield United,Aaron Ramsdale
435,Aaron Wan-Bissaka,Aaron Wan-Bissaka,Manchester United,Manchester United,Aaron Wan-Bissaka
4,Abdoulaye Doucouré,Abdoulaye Doucouré,Everton,Everton,Abdoulaye Doucouré
...,...,...,...,...,...
697,Joseph Willock,Joseph Willock,Newcastle United,NaN,NaN
709,Kyle John,Kyle John,Everton,NaN,NaN
719,Jaden Philogene-Bidace,Jaden Philogene-Bidace,Aston Villa,NaN,NaN
723,Jesurun Rak-Sakyi,Jesurun Rak-Sakyi,Crystal Palace,NaN,NaN


In [21]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    """
    :param df_1: the left table to join
    :param df_2: the right table to join
    :param key1: key column of the left table
    :param key2: key column of the right table
    :param threshold: how close the matches should be to return a match, based on Levenshtein distance
    :param limit: the amount of matches that will get returned, these are sorted high to low
    :return: dataframe with boths keys and matches
    """
    s = df_2[key2].tolist()

    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))
    df_1['matches'] = m

    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2

    return df_1
players_merged = fuzzy_merge(player_lineup, players_fl ,'player', 'player', threshold=90, limit=1)
players_merged

,player,team,name_lineup,matches
0,Marek Rodák,Fulham,Marek Rodák,Marek Rodák
1,Michael Hector,Fulham,Michael Hector,Michael Hector
2,Denis Odoi,Fulham,Denis Odoi,Denis Odoi
3,Neeskens Kebano,Fulham,Neeskens Kebano,Neeskens Kebano
4,Tom Cairney,Fulham,Tom Cairney,Tom Cairney
...,...,...,...,...
14441,Hannibal Mejbri,Manchester United,Hannibal Mejbri,Hannibal Mejbri
14442,William Thomas Fish,Manchester United,William Thomas Fish,
14523,Tim Iroegbunam,West Bromwich Albion,Tim Iroegbunam,Tim Iroegbunam
14541,Tyrese Francois,Fulham,Tyrese Francois,Tyrese Francois


In [22]:
#players_merged = players_merged[['player','Team', 'matches']]
players_merged= players_merged.sort_values(by='matches')
players_merged.reset_index(inplace=True, drop=True)
players_merged.rename(columns={"matches": "name"}, inplace=True)
players_merged



,player,team,name_lineup,name
0,Kostas Tsimikas,Liverpool,Kostas Tsimikas,
1,Kiko Casilla,Leeds United,Kiko Casilla,
2,Khanya Leshabela,Leicester City,Khanya Leshabela,
3,Carlos Vinícius,Tottenham Hotspur,Carlos Vinícius,
4,Hélder Costa,Leeds United,Hélder Costa,
...,...,...,...,...
648,Youri Tielemans,Leicester City,Youri Tielemans,Youri Tielemans
649,Yves Bissouma,Brighton & Hove Albion,Yves Bissouma,Yves Bissouma
650,Zack Steffen,Manchester City,Zack Steffen,Zack Steffen
651,Zak Brunt,Sheffield United,Zak Brunt,Zak Brunt


In [13]:
players_fl

,player,name,team
0,Aaron Connolly,Aaron Connolly,Brighton & Hove Albion
1,Aaron Cresswell,Aaron Cresswell,West Ham United
2,Aaron Mooy,Aaron Mooy,Brighton & Hove Albion
3,Aaron Ramsdale,Aaron Ramsdale,Sheffield United
4,Abdoulaye Doucouré,Abdoulaye Doucouré,Everton
...,...,...,...
23955,William Fish,William Fish,Manchester United
24064,Sylvester Jasper,Sylvester Jasper,Fulham
24183,William Osula,William Osula,Sheffield United
24237,Tim Iroegbunam,Tim Iroegbunam,West Bromwich Albion


In [42]:
out = pd.merge(players_merged,players_fl , on='name', how='left')

out = out[['player_x', 'team_x', 'name', 'team_y']]
out.rename(columns={"player_x": "player_rating", "team_x": "team_rating", "name": "player_fl", "team_y": "team_fl" }, inplace=True)
out

,player_rating,team_rating,player_fl,team_fl
0,Kostas Tsimikas,Liverpool,,NaN
1,Kiko Casilla,Leeds United,,NaN
2,Khanya Leshabela,Leicester City,,NaN
3,Carlos Vinícius,Tottenham Hotspur,,NaN
4,Hélder Costa,Leeds United,,NaN
...,...,...,...,...
672,Youri Tielemans,Leicester City,Youri Tielemans,Leicester City
673,Yves Bissouma,Brighton & Hove Albion,Yves Bissouma,Brighton & Hove Albion
674,Zack Steffen,Manchester City,Zack Steffen,Manchester City
675,Zak Brunt,Sheffield United,Zak Brunt,Sheffield United


In [26]:
out.to_excel('names_2020.xlsx')

In [54]:
name_matching = pd.read_excel('name_matching_2020.xlsx')
name_matching

,Unnamed: 0,player_rating,team_rating,player_fl,team_fl,Unnamed: 5
0,0,Kostas Tsimikas,Liverpool,Konstantinos Tsimikas,Liverpool,1.0
1,1,Kiko Casilla,Leeds United,Francisco Casilla Cortés,Leeds United,1.0
2,2,Khanya Leshabela,Leicester City,Thakgalo Leshabela,Leicester City,1.0
3,3,Carlos Vinícius,Tottenham Hotspur,Carlos Vinicius Alves Morais,Tottenham Hotspur,1.0
4,4,Hélder Costa,Leeds United,Hélder Wander Sousa de Azevedo e Costa,Leeds United,1.0
...,...,...,...,...,...,...
673,673,Yves Bissouma,Brighton & Hove Albion,Yves Bissouma,Brighton & Hove Albion,1.0
674,674,Zack Steffen,Manchester City,Zack Steffen,Manchester City,1.0
675,675,Zak Brunt,Sheffield United,Zak Brunt,Sheffield United,1.0
676,676,Çağlar Söyüncü,Leicester City,Çaglar Söyüncü,Leicester City,1.0


In [55]:
name_matching = name_matching[['player_rating', 'player_fl']]
name_matching = name_matching.drop_duplicates()
name_matching.rename(columns={"player_fl": "player"}, inplace=True)
name_matching

,player_rating,player
0,Kostas Tsimikas,Konstantinos Tsimikas
1,Kiko Casilla,Francisco Casilla Cortés
2,Khanya Leshabela,Thakgalo Leshabela
3,Carlos Vinícius,Carlos Vinicius Alves Morais
4,Hélder Costa,Hélder Wander Sousa de Azevedo e Costa
...,...,...
672,Youri Tielemans,Youri Tielemans
673,Yves Bissouma,Yves Bissouma
674,Zack Steffen,Zack Steffen
675,Zak Brunt,Zak Brunt


In [56]:
PL2020_fl ['date'] = PL2020_fl['kickoff_time'].str.slice(start=0, stop=10)
PL2020_fl

,name,position,team,xP,assists,bonus,bps,clean_sheets,creativity,element,...,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW,player,date
0,Aaron Connolly,FWD,Brighton & Hove Albion,0.5,0,0,-3,0,0.3,78,...,1,0,0,0,55,True,0,1,Aaron Connolly,2020-09-14
1,Aaron Cresswell,DEF,West Ham United,2.1,0,0,11,0,11.2,435,...,1,0,0,0,50,True,0,1,Aaron Cresswell,2020-09-12
2,Aaron Mooy,MID,Brighton & Hove Albion,0.0,0,0,0,0,0.0,60,...,0,0,0,0,50,True,0,1,Aaron Mooy,2020-09-14
3,Aaron Ramsdale,GK,Sheffield United,2.5,0,0,12,0,0.0,483,...,1,0,0,0,50,True,0,1,Aaron Ramsdale,2020-09-14
4,Abdoulaye Doucouré,MID,Everton,1.3,0,0,20,1,44.6,512,...,3,0,0,0,55,False,0,1,Abdoulaye Doucouré,2020-09-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24360,Aaron Ramsdale,GK,Sheffield United,9.0,0,2,30,1,10.0,483,...,9,13618,17381,3763,46,True,0,38,Aaron Ramsdale,2021-05-23
24361,Adrien Silva,MID,Leicester City,0.0,0,0,0,0,0.0,220,...,0,-16,0,16,44,True,0,38,Adrien Silva,2021-05-23
24362,Sidnei Tavares,MID,Leicester City,0.0,0,0,0,0,0.0,676,...,0,32,60,28,44,True,0,38,Sidnei Tavares,2021-05-23
24363,Kalvin Phillips,MID,Leeds United,6.5,0,0,17,0,12.8,204,...,6,210,789,579,49,True,1,38,Kalvin Phillips,2021-05-23


In [57]:
PL2020_fl_matched = pd.merge(PL2020_fl, name_matching , on='player', how='left')
#PL2020_fl_matched['GW']= PL2020_fl_matched['GW'].astype(float)
PL2020_fl_matched['ID'] = PL2020_fl_matched['player_rating'] + '_' + PL2020_fl_matched['date'].astype(str)
PL2020_fl_matched

,name,position,team,xP,assists,bonus,bps,clean_sheets,creativity,element,...,transfers_in,transfers_out,value,was_home,yellow_cards,GW,player,date,player_rating,ID
0,Aaron Connolly,FWD,Brighton & Hove Albion,0.5,0,0,-3,0,0.3,78,...,0,0,55,True,0,1,Aaron Connolly,2020-09-14,Aaron Connolly,Aaron Connolly_2020-09-14
1,Aaron Cresswell,DEF,West Ham United,2.1,0,0,11,0,11.2,435,...,0,0,50,True,0,1,Aaron Cresswell,2020-09-12,Aaron Cresswell,Aaron Cresswell_2020-09-12
2,Aaron Mooy,MID,Brighton & Hove Albion,0.0,0,0,0,0,0.0,60,...,0,0,50,True,0,1,Aaron Mooy,2020-09-14,NaN,NaN
3,Aaron Ramsdale,GK,Sheffield United,2.5,0,0,12,0,0.0,483,...,0,0,50,True,0,1,Aaron Ramsdale,2020-09-14,Aaron Ramsdale,Aaron Ramsdale_2020-09-14
4,Abdoulaye Doucouré,MID,Everton,1.3,0,0,20,1,44.6,512,...,0,0,55,False,0,1,Abdoulaye Doucouré,2020-09-13,Abdoulaye Doucouré,Abdoulaye Doucouré_2020-09-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24376,Aaron Ramsdale,GK,Sheffield United,9.0,0,2,30,1,10.0,483,...,17381,3763,46,True,0,38,Aaron Ramsdale,2021-05-23,Aaron Ramsdale,Aaron Ramsdale_2021-05-23
24377,Adrien Silva,MID,Leicester City,0.0,0,0,0,0,0.0,220,...,0,16,44,True,0,38,Adrien Silva,2021-05-23,NaN,NaN
24378,Sidnei Tavares,MID,Leicester City,0.0,0,0,0,0,0.0,676,...,60,28,44,True,0,38,Sidnei Tavares,2021-05-23,Sidnei Tavares,Sidnei Tavares_2021-05-23
24379,Kalvin Phillips,MID,Leeds United,6.5,0,0,17,0,12.8,204,...,789,579,49,True,1,38,Kalvin Phillips,2021-05-23,Kalvin Phillips,Kalvin Phillips_2021-05-23


In [58]:
PL2020_ratings['ID'] = PL2020_ratings['player'] + '_' + PL2020_ratings['date'].astype(str)
PL2020_ratings

,player,position,age,minutes,Team,matchweek,category,Rating,date,ID
0,Joe Bryan,LB,26-361,90.0,Fulham,1.0,Defender,30.040214,2020-09-12,Joe Bryan_2020-09-12
1,Tim Ream,CB,32-343,90.0,Fulham,1.0,Defender,33.442623,2020-09-12,Tim Ream_2020-09-12
2,Michael Hector,CB,28-055,90.0,Fulham,1.0,Defender,37.665180,2020-09-12,Michael Hector_2020-09-12
3,Denis Odoi,RB,32-108,90.0,Fulham,1.0,Defender,33.911034,2020-09-12,Denis Odoi_2020-09-12
4,Ainsley Maitland-Niles,WB,23-014,90.0,Arsenal,1.0,Defender,25.875207,2020-09-12,Ainsley Maitland-Niles_2020-09-12
...,...,...,...,...,...,...,...,...,...,...
10389,Emiliano Martínez,GK,28-263,90.0,Aston Villa,38.0,Goalkeeper,63.001016,2021-05-23,Emiliano Martínez_2021-05-23
10390,Edouard Mendy,GK,29-083,45.0,Chelsea,38.0,Goalkeeper,33.585029,2021-05-23,Edouard Mendy_2021-05-23
10391,Kepa Arrizabalaga,GK,26-232,45.0,Chelsea,38.0,Goalkeeper,26.766151,2021-05-23,Kepa Arrizabalaga_2021-05-23
10392,Łukasz Fabiański,GK,36-035,90.0,West Ham United,38.0,Goalkeeper,71.562860,2021-05-23,Łukasz Fabiański_2021-05-23


In [59]:
PL2020_matched = pd.merge(PL2020_fl_matched, PL2020_ratings , on='ID', how='left')
PL2020_matched

,name,position_x,team,xP,assists,bonus,bps,clean_sheets,creativity,element,...,ID,player_y,position_y,age,minutes_y,Team,matchweek_y,category,Rating,date_y
0,Aaron Connolly,FWD,Brighton & Hove Albion,0.5,0,0,-3,0,0.3,78,...,Aaron Connolly_2020-09-14,Aaron Connolly,FW,20-230,46.0,Brighton & Hove Albion,1.0,Forward,20.877365,2020-09-14
1,Aaron Cresswell,DEF,West Ham United,2.1,0,0,11,0,11.2,435,...,Aaron Cresswell_2020-09-12,Aaron Cresswell,LB,30-272,90.0,West Ham United,1.0,Defender,39.872009,2020-09-12
2,Aaron Mooy,MID,Brighton & Hove Albion,0.0,0,0,0,0,0.0,60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Aaron Ramsdale,GK,Sheffield United,2.5,0,0,12,0,0.0,483,...,Aaron Ramsdale_2020-09-14,Aaron Ramsdale,GK,22-123,90.0,Sheffield United,1.0,Goalkeeper,26.083953,2020-09-14
4,Abdoulaye Doucouré,MID,Everton,1.3,0,0,20,1,44.6,512,...,Abdoulaye Doucouré_2020-09-13,Abdoulaye Doucouré,CM,27-256,90.0,Everton,1.0,Midfield,31.205598,2020-09-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24376,Aaron Ramsdale,GK,Sheffield United,9.0,0,2,30,1,10.0,483,...,Aaron Ramsdale_2021-05-23,Aaron Ramsdale,GK,23-009,90.0,Sheffield United,38.0,Goalkeeper,55.932138,2021-05-23
24377,Adrien Silva,MID,Leicester City,0.0,0,0,0,0,0.0,220,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24378,Sidnei Tavares,MID,Leicester City,0.0,0,0,0,0,0.0,676,...,Sidnei Tavares_2021-05-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24379,Kalvin Phillips,MID,Leeds United,6.5,0,0,17,0,12.8,204,...,Kalvin Phillips_2021-05-23,Kalvin Phillips,DM,25-172,90.0,Leeds United,38.0,Midfield,34.090387,2021-05-23


In [62]:
PL2020_matched['Rating'] = PL2020_matched['Rating'].fillna(0)
PL2020_matched

,name,position_x,team,xP,assists,bonus,bps,clean_sheets,creativity,element,...,ID,player_y,position_y,age,minutes_y,Team,matchweek_y,category,Rating,date_y
0,Aaron Connolly,FWD,Brighton & Hove Albion,0.5,0,0,-3,0,0.3,78,...,Aaron Connolly_2020-09-14,Aaron Connolly,FW,20-230,46.0,Brighton & Hove Albion,1.0,Forward,20.877365,2020-09-14
1,Aaron Cresswell,DEF,West Ham United,2.1,0,0,11,0,11.2,435,...,Aaron Cresswell_2020-09-12,Aaron Cresswell,LB,30-272,90.0,West Ham United,1.0,Defender,39.872009,2020-09-12
2,Aaron Mooy,MID,Brighton & Hove Albion,0.0,0,0,0,0,0.0,60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN
3,Aaron Ramsdale,GK,Sheffield United,2.5,0,0,12,0,0.0,483,...,Aaron Ramsdale_2020-09-14,Aaron Ramsdale,GK,22-123,90.0,Sheffield United,1.0,Goalkeeper,26.083953,2020-09-14
4,Abdoulaye Doucouré,MID,Everton,1.3,0,0,20,1,44.6,512,...,Abdoulaye Doucouré_2020-09-13,Abdoulaye Doucouré,CM,27-256,90.0,Everton,1.0,Midfield,31.205598,2020-09-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24376,Aaron Ramsdale,GK,Sheffield United,9.0,0,2,30,1,10.0,483,...,Aaron Ramsdale_2021-05-23,Aaron Ramsdale,GK,23-009,90.0,Sheffield United,38.0,Goalkeeper,55.932138,2021-05-23
24377,Adrien Silva,MID,Leicester City,0.0,0,0,0,0,0.0,220,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN
24378,Sidnei Tavares,MID,Leicester City,0.0,0,0,0,0,0.0,676,...,Sidnei Tavares_2021-05-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN
24379,Kalvin Phillips,MID,Leeds United,6.5,0,0,17,0,12.8,204,...,Kalvin Phillips_2021-05-23,Kalvin Phillips,DM,25-172,90.0,Leeds United,38.0,Midfield,34.090387,2021-05-23


In [50]:
PL2020_check = PL2020_matched[['player_x','name' ,'date_x' ,'Team', 'matchweek_x', 'category','position_x', 'position_y', 'minutes_x', 'minutes_y', 'Rating', 'xP', 'bonus', 'bps',  'ict_index', 'influence', 'creativity','threat','total_points', 'transfers_balance', 'transfers_in', 'transfers_out', 'value', 'was_home' ]]
PL2020_check

,player_x,name,date_x,Team,matchweek_x,category,position_x,position_y,minutes_x,minutes_y,...,ict_index,influence,creativity,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home
0,Aaron Connolly,Aaron Connolly,2020-09-14,Brighton & Hove Albion,1,Forward,FWD,FW,45,46.0,...,3.4,1.2,0.3,32.0,1,0,0,0,55,True
1,Aaron Cresswell,Aaron Cresswell,2020-09-12,West Ham United,1,Defender,DEF,LB,90,90.0,...,2.2,10.4,11.2,0.0,1,0,0,0,50,True
2,Aaron Mooy,Aaron Mooy,2020-09-14,NaN,1,NaN,MID,NaN,0,NaN,...,0.0,0.0,0.0,0.0,0,0,0,0,50,True
3,Aaron Ramsdale,Aaron Ramsdale,2020-09-14,Sheffield United,1,Goalkeeper,GK,GK,90,90.0,...,1.8,18.2,0.0,0.0,1,0,0,0,50,True
4,Abdoulaye Doucouré,Abdoulaye Doucouré,2020-09-13,Everton,1,Midfield,MID,CM,90,90.0,...,6.9,20.4,44.6,4.0,3,0,0,0,55,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24566,Aaron Ramsdale,Aaron Ramsdale,2021-05-23,Sheffield United,38,Goalkeeper,GK,GK,90,90.0,...,3.2,22.0,10.0,0.0,9,13618,17381,3763,46,True
24567,Adrien Silva,Adrien Silva,2021-05-23,NaN,38,NaN,MID,NaN,0,NaN,...,0.0,0.0,0.0,0.0,0,-16,0,16,44,True
24568,Sidnei Tavares,Sidnei Tavares,2021-05-23,NaN,38,NaN,MID,NaN,0,NaN,...,0.0,0.0,0.0,0.0,0,32,60,28,44,True
24569,Kalvin Phillips,Kalvin Phillips,2021-05-23,Leeds United,38,Midfield,MID,DM,90,90.0,...,6.1,40.6,12.8,8.0,6,210,789,579,49,True


In [60]:
PL2020_matched.to_excel('check_2020.xlsx')

In [73]:
PL2020_total = PL2020_matched[['player_x','player_rating','date_x' ,'team','Team' ,'category','position_x', 'minutes_x' ,'minutes_y','Rating', 'xP', 'bonus', 'bps',  'ict_index', 'influence', 'creativity','threat','total_points', 'transfers_balance', 'transfers_in', 'transfers_out', 'value', 'was_home', 'opponent_team', 'selected' ]]
PL2020_total.rename(columns={"player_x": "player_fl","player_rating": "player" ,"date_x": "date", "Team": "team_rating","team":"team",  "position_x": "position_fl", "minutes_x": "minutes_fl","minutes_y": "minutes_rating" ,"Rating": "rating"}, inplace=True)
PL2020_total


C:\Users\giuli\AppData\Local\Temp\ipykernel_100908\3791840291.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PL2020_total.rename(columns={"player_x": "player_fl","player_rating": "player" ,"date_x": "date", "Team": "team_rating","team":"team",  "position_x": "position_fl", "minutes_x": "minutes_fl","minutes_y": "minutes_rating" ,"Rating": "rating"}, inplace=True)


,player_fl,player,date,team,team_rating,category,position_fl,minutes_fl,minutes_rating,rating,...,creativity,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,opponent_team,selected
0,Aaron Connolly,Aaron Connolly,2020-09-14,Brighton & Hove Albion,Brighton & Hove Albion,Forward,FWD,45,46.0,20.877365,...,0.3,32.0,1,0,0,0,55,True,5,32205
1,Aaron Cresswell,Aaron Cresswell,2020-09-12,West Ham United,West Ham United,Defender,DEF,90,90.0,39.872009,...,11.2,0.0,1,0,0,0,50,True,14,70232
2,Aaron Mooy,0,2020-09-14,Brighton & Hove Albion,NaN,NaN,MID,0,NaN,0.000000,...,0.0,0.0,0,0,0,0,50,True,5,2010
3,Aaron Ramsdale,Aaron Ramsdale,2020-09-14,Sheffield United,Sheffield United,Goalkeeper,GK,90,90.0,26.083953,...,0.0,0.0,1,0,0,0,50,True,20,462939
4,Abdoulaye Doucouré,Abdoulaye Doucouré,2020-09-13,Everton,Everton,Midfield,MID,90,90.0,31.205598,...,44.6,4.0,3,0,0,0,55,False,17,143040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24376,Aaron Ramsdale,Aaron Ramsdale,2021-05-23,Sheffield United,Sheffield United,Goalkeeper,GK,90,90.0,55.932138,...,10.0,0.0,9,13618,17381,3763,46,True,4,162985
24377,Adrien Silva,0,2021-05-23,Leicester City,NaN,NaN,MID,0,NaN,0.000000,...,0.0,0.0,0,-16,0,16,44,True,17,16396
24378,Sidnei Tavares,Sidnei Tavares,2021-05-23,Leicester City,NaN,NaN,MID,0,NaN,0.000000,...,0.0,0.0,0,32,60,28,44,True,17,337
24379,Kalvin Phillips,Kalvin Phillips,2021-05-23,Leeds United,Leeds United,Midfield,MID,90,90.0,34.090387,...,12.8,8.0,6,210,789,579,49,True,18,74008


In [74]:
PL2020_total['player_ID'] = PL2020_total['player'].astype(str) + PL2020_total['team'] + PL2020_total['date'].astype(str)
PL2020_total['game_ID'] = PL2020_total['team'] + PL2020_total['date'].astype(str)

C:\Users\giuli\AppData\Local\Temp\ipykernel_100908\2868901141.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PL2020_total['player_ID'] = PL2020_total['player'].astype(str) + PL2020_total['team'] + PL2020_total['date'].astype(str)
C:\Users\giuli\AppData\Local\Temp\ipykernel_100908\2868901141.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PL2020_total['game_ID'] = PL2020_total['team'] + PL2020_total['date'].astype(str)


In [75]:
PL2020_total

,player_fl,player,date,team,team_rating,category,position_fl,minutes_fl,minutes_rating,rating,...,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,opponent_team,selected,player_ID,game_ID
0,Aaron Connolly,Aaron Connolly,2020-09-14,Brighton & Hove Albion,Brighton & Hove Albion,Forward,FWD,45,46.0,20.877365,...,1,0,0,0,55,True,5,32205,Aaron ConnollyBrighton & Hove Albion2020-09-14,Brighton & Hove Albion2020-09-14
1,Aaron Cresswell,Aaron Cresswell,2020-09-12,West Ham United,West Ham United,Defender,DEF,90,90.0,39.872009,...,1,0,0,0,50,True,14,70232,Aaron CresswellWest Ham United2020-09-12,West Ham United2020-09-12
2,Aaron Mooy,0,2020-09-14,Brighton & Hove Albion,NaN,NaN,MID,0,NaN,0.000000,...,0,0,0,0,50,True,5,2010,0Brighton & Hove Albion2020-09-14,Brighton & Hove Albion2020-09-14
3,Aaron Ramsdale,Aaron Ramsdale,2020-09-14,Sheffield United,Sheffield United,Goalkeeper,GK,90,90.0,26.083953,...,1,0,0,0,50,True,20,462939,Aaron RamsdaleSheffield United2020-09-14,Sheffield United2020-09-14
4,Abdoulaye Doucouré,Abdoulaye Doucouré,2020-09-13,Everton,Everton,Midfield,MID,90,90.0,31.205598,...,3,0,0,0,55,False,17,143040,Abdoulaye DoucouréEverton2020-09-13,Everton2020-09-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24376,Aaron Ramsdale,Aaron Ramsdale,2021-05-23,Sheffield United,Sheffield United,Goalkeeper,GK,90,90.0,55.932138,...,9,13618,17381,3763,46,True,4,162985,Aaron RamsdaleSheffield United2021-05-23,Sheffield United2021-05-23
24377,Adrien Silva,0,2021-05-23,Leicester City,NaN,NaN,MID,0,NaN,0.000000,...,0,-16,0,16,44,True,17,16396,0Leicester City2021-05-23,Leicester City2021-05-23
24378,Sidnei Tavares,Sidnei Tavares,2021-05-23,Leicester City,NaN,NaN,MID,0,NaN,0.000000,...,0,32,60,28,44,True,17,337,Sidnei TavaresLeicester City2021-05-23,Leicester City2021-05-23
24379,Kalvin Phillips,Kalvin Phillips,2021-05-23,Leeds United,Leeds United,Midfield,MID,90,90.0,34.090387,...,6,210,789,579,49,True,18,74008,Kalvin PhillipsLeeds United2021-05-23,Leeds United2021-05-23


In [76]:
PL2020_total.to_csv('Data/2020-21/Players/PL2020_total.csv')